In [1]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [2]:
with open('intent.json') as file:
    data = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [3]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [4]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [5]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 9)                 153       
                                                                 
Total params: 16,697
Trainable params: 16,697
Non-trainable params: 0
____________________________________________________

In [6]:
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/500
2/2 [==============================] - 0s 4ms/step - loss: 2.1964 - accuracy: 0.1364
Epoch 2/500
2/2 [==============================] - 0s 2ms/step - loss: 2.1938 - accuracy: 0.1591
Epoch 3/500
2/2 [==============================] - 0s 3ms/step - loss: 2.1917 - accuracy: 0.1591
Epoch 4/500
2/2 [==============================] - 0s 2ms/step - loss: 2.1898 - accuracy: 0.1591
Epoch 5/500
2/2 [==============================] - 0s 3ms/step - loss: 2.1876 - accuracy: 0.1591
Epoch 6/500
2/2 [==============================] - 0s 3ms/step - loss: 2.1862 - accuracy: 0.1591
Epoch 7/500
2/2 [==============================] - 0s 2ms/step - loss: 2.1842 - accuracy: 0.1591
Epoch 8/500
2/2 [==============================] - 0s 3ms/step - loss: 2.1822 - accuracy: 0.1591
Epoch 9/500
2/2 [==============================] - 0s 3ms/step - loss: 2.1801 - accuracy: 0.1591
Epoch 10/500
2/2 [==============================] - 0s 4ms/step - loss: 2.1781 - accuracy: 0.1591
Epoch 11/500
2/2 [===========

In [7]:
# to save the trained model
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model\assets


INFO:tensorflow:Assets written to: chat_model\assets
